In [3]:
import lightgbm as lgb
import pandas as pd
import numpy as np

In [4]:
london_data = pd.read_csv("../Binary problem/Data/dataset_london.csv")

london_with_driving_lic = london_data[london_data['driving_license']==1]

london_with_car = london_with_driving_lic[london_with_driving_lic['car_ownership']>0]

ld_with_one_trip = london_with_car.groupby(by=['household_id']).sample(1)

Feature = ld_with_one_trip[['travel_mode', 'survey_year', 'dur_walking', 'dur_cycling', 'dur_pt_total', 'dur_driving', 'cost_transit', 'cost_driving_total']]

X = Feature[['survey_year', 'dur_walking', 'dur_cycling', 'dur_pt_total', 'cost_transit', 'dur_driving', 'cost_driving_total']]
y = Feature[['travel_mode', 'survey_year']]


new_mode = {'drive':0, 'pt':1, 'cycle':2,'walk':3}

y = y.replace({'travel_mode': new_mode})


X_train = X[X['survey_year']<3]
X_validate =  X[X['survey_year']==3]
y_train = y[y['survey_year']<3]
y_validate = y[y['survey_year']==3]

X_train = X_train.drop(columns=['survey_year'])
X_validate = X_validate.drop(columns=['survey_year'])
y_train = y_train.drop(columns=['survey_year'])
y_validate = y_validate.drop(columns=['survey_year'])

In [75]:
train_data = lgb.Dataset(X_train.iloc[3:5, :], label=y_train.iloc[3:5], free_raw_data=False)
validation_data = lgb.Dataset(X_validate, label=y_validate, reference= train_data, free_raw_data=False)

In [96]:
param_ld_unc = {'max_depth': 1, 
         'num_boost_round': 2, 
         'objective':'multiclass',
         'learning_rate': .1,
         'verbosity': 2,
         'num_classes': 4,
         'min_data_in_leaf': 1,
         'min_sum_hessian_in_leaf':0
        }

In [97]:
lightgbm_ld_unc = lgb.train(param_ld_unc, train_data)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.500000
[LightGBM] [Debug] init for col-wise cost 0.000004 seconds, init for row-wise cost 0.000419 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000445 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 1
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1


c:\Users\nicol\Anaconda3\envs\lightgbm\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [98]:
lightgbm_ld_unc.predict(X_train.iloc[3:5], raw_score = True)

array([[ -0.41258582, -34.53877639, -34.53877639,  -0.97370854],
       [ -0.97370854, -34.53877639, -34.53877639,  -0.41258582]])

In [79]:
X_train.iloc[3:5]

p1 0.71  g1 0.29 h1 0.2059
p2 0
p3 0
p4 0.29  g4 -0.29 h4 

,dur_walking,dur_cycling,dur_pt_total,cost_transit,dur_driving,cost_driving_total
20,0.598333,0.183333,0.438611,0.0,0.117222,0.41
25,0.165000,0.062778,0.135000,1.5,0.049167,0.10


In [65]:
y_train.iloc[:5]

,travel_mode
2,0
11,0
14,0
20,0
25,3


In [ ]:
'leaf_value': 0.05685279708831137
'leaf_weight': 0.3333333432674408

In [92]:
model = lightgbm_ld_unc.dump_model()
model

{'name': 'tree',
 'version': 'v3',
 'num_class': 4,
 'num_tree_per_iteration': 4,
 'label_index': 0,
 'max_feature_idx': 5,
 'objective': 'multiclass num_class:4',
 'average_output': False,
 'feature_names': ['dur_walking',
  'dur_cycling',
  'dur_pt_total',
  'cost_transit',
  'dur_driving',
  'cost_driving_total'],
 'monotone_constraints': [],
 'feature_infos': {'cost_transit': {'min_value': 0,
   'max_value': 1.5,
   'values': []}},
 'tree_info': [{'tree_index': 0,
   'num_leaves': 2,
   'num_cat': 0,
   'shrinkage': 1,
   'tree_structure': {'split_index': 0,
    'split_feature': 3,
    'split_gain': 1.5,
    'threshold': 1.0000000180025095e-35,
    'decision_type': '<=',
    'default_left': True,
    'missing_type': 'None',
    'internal_value': -0.693147,
    'internal_weight': 0,
    'internal_count': 2,
    'left_child': {'leaf_index': 0,
     'leaf_value': -0.24314719397099133,
     'leaf_weight': 0.3333333432674408,
     'leaf_count': 1},
    'right_child': {'leaf_index': 1,
 

In [51]:
new_p1 = np.exp(-0.5*0.3817384822141626)
new_p2 = np.exp(-0.5*1.6746763792421082)
new_p3 = np.exp(-0.5*3.7242164796099657)
new_p4 = np.exp(-0.5*0.5907802951783105)

new_sum = new_p1 + new_p2 + new_p3 + new_p4

newp1 = new_p1/new_sum
newp2 = new_p2/new_sum
newp3 = new_p3/new_sum
newp4 = new_p4/new_sum

newp1

0.38275133506085757

In [45]:
p4

0.11924539168303395

In [81]:
summed = np.exp(-0.693147) + np.exp(-34.538776) + np.exp(-34.538776) + np.exp(-0.693147)
p1 = np.exp(-0.693147)/summed
p2 = np.exp(-34.538776)/summed
p3 = np.exp(-34.538776)/summed
p4 = np.exp(-0.693147)/summed

In [82]:
[p1, p2, p3, p4]

[0.499999999999999,
 1.0000002143507624e-15,
 1.0000002143507624e-15,
 0.499999999999999]

In [ ]:
g1 = 

In [34]:
preds = np.repeat([[p1], [p2], [p3], [p4]], 7103, axis = 1).T

In [35]:
preds

array([[0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       ...,
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539]])

In [36]:
c_entr = 0
for i, l in enumerate(y_train['travel_mode']):
    c_entr += np.log(preds[i, l])


In [110]:
labels1 = (y_train).astype(int)

In [155]:
g1 = labels1-p1
h1 = np.repeat(p1 * (1 - p1), 7103)

In [156]:
g_split1 = g1[X_train['cost_driving_total']<7.070000000000001]
h_split1 = h1[X_train['cost_driving_total']<7.070000000000001]
g_split1 = g_split1['travel_mode'].tolist()
h_split1 = h_split1.tolist()
g_split2 = g1[X_train['cost_driving_total']>7.070000000000001]
h_split2 = h1[X_train['cost_driving_total']>7.070000000000001]
g_split2 = g_split2['travel_mode'].tolist()
h_split2 = h_split2.tolist()



In [157]:
np.size(g_split1)

6381

In [172]:
np.sum(g1)**2/(722+6381)

travel_mode    1.552188
dtype: float64

In [173]:
(np.sum(g_split1)**2/6381 + np.sum(g_split2)**2/722)

165.21321820703662

In [161]:
weights = [g^2/h for g, h in zip(g_split1, h_split1)]

TypeError: unsupported operand type(s) for ^: 'float' and 'float'

In [162]:
h_split1

[0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.2342825

In [163]:
np.mean(weights)

-0.15484219236329722

In [165]:
np.sum(h_split1)

1494.9566623342357

In [82]:
h_split2

[0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515]